In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/nlp_liflow/

/content/drive/My Drive/nlp_liflow


In [ ]:
!ls

자연어처리_1.ipynb		  item_all_doc2vec_k-means.csv
자연어처리_2.ipynb		  ko.bin
all_df_with_tokens.csv		  list_to_csv.csv
all_sen.csv			  nlp_tokenizer.ipynb
cc.ko.300.bin.gz		  스포츠_sen.csv
df_all				  sen.csv
df_all.csv			  sen_preprocessig.csv
df_all_GloVe.csv		  스포츠_sen_prerprocessing.csv
df_all_tfidf.csv		  sentence_Summarizer.ipynb
doc2vec_100.model		  textrank
feature.pkl			  textrank_git
feature.pkl.mtx			  tfidftransformer.pkl
first_embedding_clustering.model  자연어처리_tomotopy_HDPModel.ipynb
glove_model_100.model		  자연어처리_tomotopy.ipynb
glove_model.model		  자연어처리_tomotopy_modeling.ipynb
glove_word_dict_128.pickle	  vectorizer.pkl
item_1.csv			  word2vec_model
item_all.csv


In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2022-11-26 13:56:20--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2022-11-26 13:56:21--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdd4615bbf7b186a58b3534935d.dl.dropboxusercontent.com/cd/0/get/BxevBBNgBvRjwhCxxzoKNSeg4yCNVellnIwPdKqOkW2_BTCRwUJFhXFfLrNxAT-Mc7_ARf2UawuabzcnTuq_xkXCWW2y98Q6j4v033RJEa-DQWqd7eSF-gZ-bDHJBZeqhZkF6vZEjJaHdChwMg440yWQZpF5eXtMKdWvaeIXk3pDxFuxfo-cUaUCvJNMB2J8HR8/file?dl=1# [following]
--2022-11-26 13:56:22--  https://ucdd4615bbf7b186a58b3534935d.dl.dropboxusercontent.com/cd/0/get/BxevBBNgB

In [ ]:
!pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.4 MB 1.2 MB/s 
     |████████████████████████████████| 131 kB 56.7 MB/s 


In [ ]:
!pip install glove_python_binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 4.0 MB/s 


In [ ]:
import textrank_git
from textrank_git import rank
from textrank_git import sentence
from textrank_git import summarizer
from textrank_git import utils
from textrank_git import word
import re
import kss
import pandas as pd
from konlpy.tag import Mecab
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim import models
import gensim
import logging
import os
from collections import namedtuple, defaultdict
from collections.abc import Iterable
from timeit import default_timer
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
from numpy import zeros, float32 as REAL, vstack, integer, dtype
import numpy as np
from gensim import utils, matutils  # utility fnc for pickling, common scipy operations etc
from gensim.utils import deprecated
from gensim.models import Word2Vec # noqa: F401
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score


KeyboardInterrupt: ignored

In [ ]:
main_df = pd.read_csv('all_df_with_tokens.csv',index_col = 0)

In [ ]:
main_df['convrsThema'] = main_df['convrsThema'].str.replace('코로나/생활', '투병')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('메르스', '투병')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('축구선수', '축구')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 문학 ', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('독서', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('문학', '책')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('집값', '부동산')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('분양', '부동산')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직업', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('좋은글,사자성어,좋은글귀,등등 읽으면 도움이되는 글들..', '좋은글')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('국제관계', '정치')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('제테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시사/재테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 경제/재테크 ', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 경제/재태크 ', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 자연/휴양지 ', '자연/여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자동차보험', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교통사고', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('운전', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 자동차 ', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀취업', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀직장', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀입대', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀교육', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀결혼', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 취업', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 교육', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자녀 결혼', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상생활', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상 이야기', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('일상대화', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#일상', '일상')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('이사', '이사/이민')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('이민', '이사/이민')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('먹거리', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('먹거리/건강', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 먹거리 ', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('육아', '육아')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('양육/교육', '육아')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교육', '교육')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('운동', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 스포츠/레저 ', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화관', '영화관')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 영화 ', '영화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('연예인', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 방송/연예 ', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 연애/결혼 ', '연애')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행계획', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여가생활', '여가')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시댁살이', '시집살이')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('시댁', '시집살이')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('말씀나누기', '대화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 만화 ', '만화')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('결혼생활', '결혼')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('남편', '결혼')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강/다이어트', '다이어트')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 건강/다이어트 ', '다이어트')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('노후준비', '노후')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('노후대책', '노후')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('넷플릭스', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('TV', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('드라마', '방송')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 군대 ', '군대')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강/교육', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('가족건강', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('가정', '가족')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#인생', '인생')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('반려동물,강아지', '반려동물')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 국가/지역 ', '국가/지역')
main_df['convrsThema'] = main_df['convrsThema'].str.replace(' 게임 ', '게임')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식', '건강식품')

main_df['convrsThema'] = main_df['convrsThema'].str.replace('수영', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('볼링', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('축구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('야구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('배구', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('골프', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직장상', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('#직장상', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('직장화', '직장')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('피시방', '게임')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('요리', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('밥', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식품품', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강식품', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('식습관', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('음식점', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('음식/건강', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영양제', '음식')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('제주도', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('휴양지', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('등산', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('캠핑', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여가', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('게읻트볼', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자전거', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스쿠버다이빙', '취미생활')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('해외여행', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('건강검진', '건강')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('연애/결혼', '연애')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('육아', '자녀')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스포츠/레저', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('재태크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('부동산', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('경제', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('아파트', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('재개발', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('돈', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('경제/재테크', '재테크')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('방송', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('영화관', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('방송/연예/연예', '방송/연예')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('책', '교육')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('교통사고/보험/보험', '교통사고/보험')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('자동차면허', '자동차')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('벌초', '제사')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('장례', '제사')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('학창시절', '친구')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('스포츠/레저', '스포츠')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행/건강', '여행')
main_df['convrsThema'] = main_df['convrsThema'].str.replace('여행/자녀', '자녀')

In [ ]:
import operator
files = main_df.loc[operator.or_(main_df['colctUnitCode']=="음성수집 도구" , main_df['colctUnitCode']=="스튜디오"), ]

In [ ]:
files = files.loc[files['convrsThema']=="스포츠", "sentence"]

In [ ]:
files

In [ ]:
mecab = Mecab()

def tokenizer(raw, pos=["NNG","NNP","VV","VA"]): 
  return [word for word, tag in mecab.pos(raw) if len(word) > 1 and tag in pos]

def duplicate_drop(df):
  df_a = df.apply(lambda a : tokenizer(a))
  df_a= df_a.drop_duplicates(keep='first')
  idx = df_a.index
  return df.loc[idx]

def preprocessing(sents):
  remain_pattern = re.compile(r'\([^)]*\)')
  hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')

  ws_replace = re.sub(r'\xa0', r'', sents)
  text_rm = remain_pattern.sub(r'', ws_replace)
  text = hangul.sub(r'', text_rm)
      
  return text

def whitespace_drop(sents):
  sents = sents.replace(" ","")

  return sents

In [ ]:
files = files.apply(lambda a : preprocessing(a))
files_wsd = files.apply(lambda a : whitespace_drop(a))

In [ ]:
files_wsd = files_wsd.drop_duplicates(keep='first')

In [ ]:
idx = files_wsd.index

In [ ]:
files = files.loc[idx]
files

In [ ]:
files = duplicate_drop(files)

In [ ]:
files

In [ ]:
files.to_csv("스포츠_sen.csv") 

##GloVe 사용한 임베딩

In [ ]:
token = []
for i in files:
  token.append(tokenizer(i))

In [ ]:
token

In [ ]:
import pickle
import re
import numpy as np
from scipy.sparse.dia import dia_matrix
import joblib
from glove import Glove
from glove import Corpus

차원 100으로 진행한 임베딩 결과

In [ ]:
corpus = Corpus() 

# 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
corpus.fit(token, window=5)
glove = Glove(no_components=100, learning_rate=0.05)

# 학습에 이용할 쓰레드의 개수는 4로 설정, 에포크는 20.
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

In [ ]:
glove.save("glove_model_100.model")

In [ ]:
glove_model_100 = Glove.load('glove_model_100.model')

In [ ]:
# word dict 생성
word_dict = {}
for word in  glove_model_100.dictionary.keys():
    word_dict[word] = glove_model_100.word_vectors[glove_model_100.dictionary[word]]
print('[Success !] Lengh of word dict... : ', len(word_dict))

# save word_dict
with open('glove_word_dict_128.pickle', 'wb') as f:
    pickle.dump(word_dict, f)
print('[Success !] Save word dict!...')

In [ ]:
def sent2vec_glove(tokens, embedding_dim=100):
    '''문장 token 리스트를 받아서 임베딩 시킨다.'''
  
    size = len(tokens)
    matrix = np.zeros((size, embedding_dim))
    word_table = word_dict     # glove word_dict

    for i, token in enumerate(tokens):
        vector = np.array([
            word_table[t] for t in token
            if t in word_table
        ])

        if vector.size != 0:
            final_vector = np.mean(vector, axis=0)
            matrix[i] = final_vector

    return matrix

In [ ]:
matrix_100 = sent2vec_glove(token)

In [ ]:
doc_vec_GloVe_100  = pd.DataFrame(columns=['convrs','trained_vector','token'])

for text, i, t in zip(files,matrix_100,token):
  new_row = pd.DataFrame([[text,i,t]],columns =  doc_vec_GloVe_100 .columns)
  doc_vec_GloVe_100 = pd.concat([doc_vec_GloVe_100 , new_row], ignore_index = True)
  print(f"text: {text}, tokens: {t}")
  print(f"trained_doc_vec: {i}")

In [ ]:
doc_vec_GloVe_100

In [ ]:
from sklearn import preprocessing
print("##"*30)
print("K-Means Clustering")
print("##"*30)
# 시도해 볼 cluster 수
cluster_number = range(5,101)
distortions = []
processed_data = doc_vec_GloVe_100.copy()
scaler = preprocessing.MinMaxScaler()
scaler.fit_transform(matrix_100)
#scaler.fit_transform(processed_data[['x', 'y']])

# 모델 학습
for k in cluster_number:
    Clustering_Method = KMeans(n_clusters=k, random_state = 0)
    #X = model.docvecs.vectors_docs 
    Clustering_Method.fit(matrix_100)
    distortions.append(Clustering_Method.inertia_)
    
# Elbow Curve 시각화
fig = plt.figure(figsize=(15, 5))
plt.plot(cluster_number, distortions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()

In [ ]:
from math import*
def euclidean_distance(x, y):
    return sqrt(sum(pow(a - b, 2) for a, b in zip(x, y)))

In [ ]:
Clustering_Method = KMeans(n_clusters=15, random_state=0)
X = matrix_100 # document vector 전체를 가져옴. 
#processed_data = tsnedata.copy()
scaler = preprocessing.MinMaxScaler()
scaler.fit_transform(X)
Clustering_Method.fit(X)# fitting 
count = 0
#Clustering_Method.cluster_centers_
cluster_dict = {i:[] for i in range(0, 15)}
for text, label in zip(range(len(doc_vec_GloVe_100)), Clustering_Method.labels_):
    a = []
    a.append(doc_vec_GloVe_100['convrs'].loc[text])
    #a.append([doc_vec['tsne_x'].loc[text],doc_vec['tsne_y'].loc[text]])
    a.append([doc_vec_GloVe_100['token'].loc[text]])
    a.append([doc_vec_GloVe_100['trained_vector'].loc[text]])
    cluster_dict[label].append(a)
    

item_all = pd.DataFrame(columns = ['cluster','convrs','vec','distance','token'])
for (label, lst), y in zip(cluster_dict.items(),Clustering_Method.cluster_centers_):
    item = pd.DataFrame(columns = ['cluster','convrs','vec','distance','token'])
    print(f"Cluster {label}")
    for x in lst:
          token = str(x[1][0])
          vec = str(x[2][0])
          #print(x[2])
          #print(y)
          print(x[0])
          distance = euclidean_distance(x[2][0],y)
          new_df = pd.DataFrame({'cluster':label,
                                'vec': vec,
                                'token': token,
                                'convrs':x[0],
                                'distance': distance},index = [0])
          count+=1
          item = pd.concat([item,new_df], ignore_index = True)
    item = item.sort_values('distance')
    item_all = pd.concat([item_all,item], ignore_index = True)
    print("--"*30)
print(count)
print("##"*20)

In [ ]:
item_all

In [ ]:
item_all.to_csv("item_all_glove_k-means.csv")

In [ ]:
x_train= []
y_train=[]
for q,label in zip(item_all['vec'], item_all['cluster'])  :
    x_train.append(q)
    y_train.append(label)
x_train= np.array(x_train)
y_train= np.array(y_train)
x_train[0]

'[-1.26730001e-03 -6.48304205e-05  2.71606257e-03 -7.88230164e-04\n -9.39257717e-04  1.17161836e-03  7.80199578e-04  2.83759983e-03\n  1.15035624e-03 -3.56600607e-03 -5.79755357e-04 -1.49690942e-03\n -9.28900550e-04  6.62274010e-04 -6.91821336e-04  2.42688425e-04\n  2.11175854e-03 -2.21879198e-03 -3.58314200e-04  9.40893508e-04\n -1.82940817e-03  1.20405092e-03  1.32704442e-03 -1.77532053e-03\n  4.20854427e-04  1.39012539e-03  4.37560975e-03 -1.55824909e-04\n  1.39237992e-04 -2.39435041e-03  1.30143822e-03 -9.35896175e-04\n -1.86506515e-04 -4.23016126e-04  7.05778625e-04 -2.31895407e-03\n -1.82957051e-03  3.70559269e-03  1.77907564e-03 -1.78767766e-03\n  2.54663531e-03  7.68447865e-05 -1.88261071e-03 -3.65241281e-04\n -5.46628784e-04  1.11296794e-04 -2.36442877e-03  3.37171857e-03\n  7.07400023e-05  7.50230736e-04  1.18082542e-03  1.44630576e-03\n  5.22064081e-04  3.83231153e-03  2.82178729e-04 -2.73581183e-03\n -1.99151709e-04  7.00889224e-04  7.80949754e-04 -1.64610670e-03\n -9.65363

In [ ]:
from keras.preprocessing import sequence
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train) # one_hot으로 변형
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)